In [ ]:
import cv2
import numpy as np
import pandas as pd
#noisy - modified from Shubham Pachori on stackoverflow
def noisy(image, noise_type, sigma):
    if noise_type == "gauss":
        row,col = image.shape
        mean = 0
        gauss = np.random.normal(mean,sigma,(row,col))
        gauss = gauss.reshape(row,col)
        noisy = image + gauss
        return noisy
    elif noise_type == "s&p":
        row,col = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
                  for i in image.shape]
        out[coords] = 1
        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
                  for i in image.shape]
        out[coords] = 0
        return out
    elif noise_type == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_type =="speckle":
        row,col = image.shape
        gauss = np.random.randn(row,col)
        gauss = gauss.reshape(row,col)
        noisy = image + image * gauss
        return noisy
motherboard = cv2.imread("motherboard-gray.png",0)
template = cv2.imread("template.png",0)
w, h = template.shape[::-1]
val=[]
for i in range (6):
    val0=[]
    for j in range (11):
        motherboard = cv2.imread("motherboard-gray.png",0)
        noisy_image = np.uint8(noisy(motherboard, 'gauss', j))
        if i ==0 :
            noisy_blur = cv2.GaussianBlur(noisy_image,(5,5), i)
        else:
            noisy_blur = cv2.GaussianBlur(noisy_image,((6*i)+1,(6*i)+1), i)
        res = cv2.matchTemplate(noisy_blur,template,cv2.TM_CCORR_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        val0.append(max_val)
        loc = np.where( res >= max_val)
        for pt in zip(*loc[::-1]):
            cv2.rectangle(motherboard, pt, (pt[0] + w, pt[1] + h), (255,0,0), 2)
    val.append(val0)
motherboard = cv2.resize(motherboard,None,fx=0.5,fy=0.5)
cv2.imshow('res.png',motherboard)
cv2.imwrite('res.png',motherboard)
res = cv2.resize(res,None,fx=0.5,fy=0.5)
cv2.imshow('Match_Template.png',res)
noisy_image = cv2.resize(noisy_image,None,fx=0.5,fy=0.5)
cv2.imshow('noisy.png',noisy_image)
noisy_blur = cv2.resize(noisy_blur,None,fx=0.5,fy=0.5)
cv2.imshow('Smoothned.png',noisy_blur)
cv2.imwrite('Smoothned.png',noisy_blur)
cv2.imwrite('Match_Template.png',res)
df = pd.DataFrame(np.transpose(val))
df.to_excel(excel_writer = "test.xlsx")
cv2.waitKey(0)
cv2.destroyAllWindows()